In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI
import httpx
import os

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
http_client=httpx.Client(verify=False)
client = OpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    http_client=http_client,
    api_key=os.environ['LLM_API']
)

In [3]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.read()
len(required_prompt)

56548

In [6]:
streaming = True
max_output_tokens = 100

# Read the content of 'output.txt' as the required prompt
# with open('../output.txt', 'r') as f:
#     required_prompt = f.read()

# Available Models list
available_models = ["mixtral-8x7b-instruct-v01", "llamaguard-7b", "gemma-7b-it", "mistral-7b-instruct-v02", "phi-2", "llama-2-70b-chat", "phi-3-mini-128k-instruct", "llama-3-8b-instruct"]

# Let's select the model from the available list
model_selected = available_models[0]


print(f"Model: {model_selected}")
completion = client.completions.create(
    model=model_selected,
    max_tokens=max_output_tokens,
    prompt="Hi, I am bob, tell me about yourself.",
    stream=streaming
)

if streaming:
    for chunk in completion:
        print(chunk.choices[0].text, end='')
else:
    print(completion.choices[0].text)


Model: mixtral-8x7b-instruct-v01


Hi bob, I am a computer algorithm designed to interact with humans. I don't have personal experiences or feelings, but I am programmed to understand and respond to human language in a way that simulates conversation. How can I help you today? Is there something specific you would like to talk about?

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 7f4a3b7a-c43e-4d4d-8d02-32a536c98eae not found for run b5e7127d-d482-4809-af11-4fa954e8ebde. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. How can I help you today? Is there something you would like to know or talk about? I'm here to answer any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5e7127d-d482-4809-af11-4fa954e8ebde-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62})

In [14]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    )
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

llm = ChatOpenAI(base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 01dee4e9-428e-4a98-9585-bec7cf7ef13b not found for run fb4faafb-4435-4391-aa85-d8c13d9b8e3b. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. Is there something specific you would like to talk about or ask me? I'm here to help with any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb4faafb-4435-4391-aa85-d8c13d9b8e3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59})

In [15]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    "Hi, I'm Bob. What's your name?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Nice to meet you! What do you know about Python?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")

# Third interaction with a different session
print("\nThird interaction (different session):")
response3 = chain.invoke(
    "Hello, I'm Alice. Can you tell me about the weather?",
    config={"configurable": {"session_id": "2"}},
)
print("Assistant:", response3.content)

# Check history for both sessions
print_chat_history("1")
print_chat_history("2")

Model: mixtral-8x7b-instruct-v01


Parent run c7cca3da-4cb6-472b-9339-f6b837bd0f2d not found for run 8ce21e30-5c35-4f4e-8950-b32f3cddf423. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 5207cc8d-87b3-4ca8-aa61-2aff9e8de81c not found for run 64872ab4-ee3d-409e-8be6-b81d69112391. Treating as a root run.


Assistant:  Hello Bob, I'm an assistant. I don't have a personal name as I'm here to help you with any questions or information you need. How can I assist you today?

Chat History for session 1:
human: Hi, I'm Bob. What's your name?
ai:  Hello Bob, I'm an assistant. I don't have a personal name as I'm here to help you with any questions or information you need. How can I assist you today?

Second interaction:


Parent run a537d0d0-8aa3-4b3b-b6e7-c0355b3f4b62 not found for run f6216de6-c11b-42bb-8a9b-026fca68710d. Treating as a root run.


Assistant:  Hello Bob, nice to meet you too! I can tell you that Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991. It is designed with an emphasis on code readability, and its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It has a large standard library that is available for use without additional installation, which includes modules for working with files, interacting with operating systems, and handling common data structures.

Python is widely used in various domains, such as web development, scientific computing, data analysis, artificial intelligence, machine learning, and automation. It also has a large and active community that contributes to its development and provides support to users.

Chat History for session 1